In [1]:
!pip install torch-geometric
!pip install sentence_transformers

  Obtaining dependency information for torch-geometric from https://files.pythonhosted.org/packages/65/4e/6f9a75548a93fedcd4514ae2de9bee1e91bade6b73252b4da32f0e42ac52/torch_geometric-2.4.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=d13612361939d49dfbe40f82d12ec46f3420bc4a6be8cd5b90237d8ab8fea03a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [37]:
import torch
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.datasets import MovieLens
from torch_geometric.nn import GATConv, SAGEConv, to_hetero
import torch_geometric.transforms as T

import numpy as np

In [3]:
dataset_path = '/tmp/'
dataset = MovieLens(root=dataset_path)

Extracting /tmp/raw/ml-latest-small.zip
Processing...


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/305 [00:00<?, ?it/s]

Done!


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data = dataset[0].to(device)

# Add user node features for message passing:
data['user'].x = torch.eye(data['user'].num_nodes, device=device)
del data['user'].num_nodes

# Add a reverse ('movie', 'rev_rates', 'user') relation for message passing:
data = T.ToUndirected()(data)
del data['movie', 'rev_rates', 'user'].edge_label  # Remove "reverse" label.

# Perform a link-level split into training, validation, and test edges:
train_data, val_data, test_data = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    neg_sampling_ratio=0.0,
    edge_types=[('user', 'rates', 'movie')],
    rev_edge_types=[('movie', 'rev_rates', 'user')],
)(data)

In [12]:
weight = torch.bincount(train_data['user', 'movie'].edge_label)
weight = weight.max() / weight

weight

tensor([26.2224,  7.6305,  2.6939,  1.0637,  1.0000,  2.6983], device='cuda:0')

In [18]:
def weighted_mse_loss(pred, target, weight=None):
    weight = 1. if weight is None else weight[target].to(pred.dtype)
    return (weight * (pred - target.to(pred.dtype)).pow(2)).mean()

## Задание

1) Подберите оптимальные параметры для сети из примера выше(2 балла)

2) Попробуйте вместо GraphSage модуль Graph Attention и также подберите оптимальные параметры  (2 балла)


## Задание 1

Добавим еще один линейный слой в EdgeDecoder

In [29]:
class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, hidden_channels)
        self.lin3 = Linear(hidden_channels, 1)

    def forward(self, z_dict, edge_label_index):
        row, col = edge_label_index
        z = torch.cat([z_dict['user'][row], z_dict['movie'][col]], dim=-1)

        z = self.lin1(z).relu()
        z = self.lin2(z).relu()
        z = self.lin3(z)
        return z.view(-1)

In [30]:
class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.encoder = GNNEncoder(hidden_channels, hidden_channels)
        self.encoder = to_hetero(self.encoder, data.metadata(), aggr='sum')
        self.decoder = EdgeDecoder(hidden_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        z_dict = self.encoder(x_dict, edge_index_dict)
        return self.decoder(z_dict, edge_label_index)

def train():
    model.train()
    optimizer.zero_grad()
    pred = model(train_data.x_dict, train_data.edge_index_dict,
                 train_data['user', 'movie'].edge_label_index)
    target = train_data['user', 'movie'].edge_label
    loss = weighted_mse_loss(pred, target, weight)
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test(data):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict,
                 data['user', 'movie'].edge_label_index)
    pred = pred.clamp(min=0, max=5)
    target = data['user', 'movie'].edge_label.float()
    rmse = F.mse_loss(pred, target).sqrt()
    return float(rmse)

Увеличим число скрытых каналов, число эпох, немного уменьшим lr оптимизатора

In [25]:
model = Model(hidden_channels=52).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0075)


for epoch in range(1, 351):
    loss = train()
    train_rmse = test(train_data)
    val_rmse = test(val_data)
    test_rmse = test(test_data)
    if epoch % 50 == 0:
        print(f'Epoch: {epoch}, Loss: {loss:.4f}, Train: {train_rmse:.4f}, '
              f'Val: {val_rmse:.4f}, Test: {test_rmse:.4f}')

Epoch: 50, Loss: 3.7455, Train: 1.1498, Val: 1.1483, Test: 1.1554
Epoch: 100, Loss: 3.0354, Train: 1.1114, Val: 1.1507, Test: 1.1458
Epoch: 150, Loss: 2.7739, Train: 1.0692, Val: 1.1209, Test: 1.1221
Epoch: 200, Loss: 2.3681, Train: 1.0287, Val: 1.1001, Test: 1.1096
Epoch: 250, Loss: 2.2822, Train: 1.0498, Val: 1.1460, Test: 1.1482
Epoch: 300, Loss: 1.9758, Train: 0.9600, Val: 1.0831, Test: 1.0932
Epoch: 350, Loss: 1.8657, Train: 0.9121, Val: 1.0535, Test: 1.0636


Получилось добиться результата лучше 1.1

## Задание 2

Перепишем GNNEncoder

In [44]:
class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, num_heads):
        super().__init__()
        self.conv1 = GATConv((-1, -1), hidden_channels, heads=num_heads, dropout=0.6, add_self_loops=False)
        self.conv2 = GATConv(hidden_channels * num_heads, out_channels, heads=1, dropout=0.6, add_self_loops=False)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, hidden_channels)
        self.lin3 = Linear(hidden_channels, 1)

    def forward(self, z_dict, edge_label_index):
        row, col = edge_label_index
        z = torch.cat([z_dict['user'][row], z_dict['movie'][col]], dim=-1)

        z = self.lin1(z).relu()
        z = self.lin2(z).relu()
        z = self.lin3(z)
        return z.view(-1)

In [45]:
class Model(torch.nn.Module):
    def __init__(self, hidden_channels, num_heads):
        super().__init__()
        self.encoder = GNNEncoder(hidden_channels, hidden_channels, num_heads)
        self.encoder = to_hetero(self.encoder, data.metadata(), aggr='sum')
        self.decoder = EdgeDecoder(hidden_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        z_dict = self.encoder(x_dict, edge_index_dict)
        return self.decoder(z_dict, edge_label_index)

def train():
    model.train()
    optimizer.zero_grad()
    pred = model(train_data.x_dict, train_data.edge_index_dict,
                 train_data['user', 'movie'].edge_label_index)
    target = train_data['user', 'movie'].edge_label
    loss = weighted_mse_loss(pred, target, weight)
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test(data):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict,
                 data['user', 'movie'].edge_label_index)
    pred = pred.clamp(min=0, max=5)
    target = data['user', 'movie'].edge_label.float()
    rmse = F.mse_loss(pred, target).sqrt()
    return float(rmse)

In [51]:
model = Model(hidden_channels=32, num_heads=1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)


for epoch in range(1, 501):
    loss = train()
    train_rmse = test(train_data)
    val_rmse = test(val_data)
    test_rmse = test(test_data)
    if epoch % 50 == 0:
        print(f'Epoch: {epoch}, Loss: {loss:.4f}, Train: {train_rmse:.4f}, '
              f'Val: {val_rmse:.4f}, Test: {test_rmse:.4f}')

Epoch: 50, Loss: 6.5037, Train: 1.3019, Val: 1.3412, Test: 1.3397
Epoch: 100, Loss: 5.6251, Train: 1.4281, Val: 1.4483, Test: 1.4442
Epoch: 150, Loss: 5.0196, Train: 1.3554, Val: 1.3720, Test: 1.3772
Epoch: 200, Loss: 4.7286, Train: 1.2571, Val: 1.2717, Test: 1.2726
Epoch: 250, Loss: 4.5608, Train: 1.1129, Val: 1.1231, Test: 1.1220
Epoch: 300, Loss: 4.1985, Train: 1.1592, Val: 1.1668, Test: 1.1622
Epoch: 350, Loss: 3.9516, Train: 1.0996, Val: 1.1120, Test: 1.1082
Epoch: 400, Loss: 3.8310, Train: 1.0310, Val: 1.0452, Test: 1.0439
Epoch: 450, Loss: 3.7637, Train: 1.0905, Val: 1.1034, Test: 1.0983
Epoch: 500, Loss: 3.7187, Train: 0.9989, Val: 1.0077, Test: 1.0130
